In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.append(f'/content/drive/My Drive/')
import os
os.chdir(f'/content/drive/My Drive/')

In [ ]:
!pip install -r requirements.txt

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import models
import transformers

In [ ]:
def generate_sentence_model_using_bert():
    transformers.BertTokenizer = transformers.BertJapaneseTokenizer
    transformer = models.BERT('cl-tohoku/bert-base-japanese-whole-word-masking')
    pooling = models.Pooling(transformer.get_word_embedding_dimension(), 
                            pooling_mode_mean_tokens=True, 
                            pooling_mode_cls_token=False, 
                            pooling_mode_max_tokens=False)
    model = SentenceTransformer(modules=[transformer, pooling])

    return model

In [ ]:
import scipy.spatial
import pickle

def compute_similarity(model, corpus, target_sentences, recommend_sentences):
    if os.path.exists("/content/drive/My Drive/corpus/embedded_livedoornews.pickle"):
        corpus_embeddings = pickle.load(open("/content/drive/My Drive/corpus/embedded_livedoornews.pickle", "rb"))
    else:
        corpus_embeddings = model.encode(corpus)
        pickle.dump(corpus_embeddings, open("/content/drive/My Drive/model/bert_model_for_livedoornews.pickle", "wb"))

    target_embeddings = model.encode(target_sentences)

    for target_sentence, target_embedding in zip(target_sentences, target_embeddings):
        distances = scipy.spatial.distance.cdist([target_embedding], corpus_embeddings, 
                                                 metric="cosine")[0]

        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])

        print("\n\n======================\n\n")
        print("Target sentence : {}".format(target_sentence))
        print("\nTop {} most similar sentences:".format(recommend_sentences))

        for idx, distance in list(reversed(results[0:recommend_sentences])):
            print("{} \n Similarity: {:.4f}%".format(corpus[idx].strip(), distance * 100 / 2))

In [ ]:
model = generate_sentence_model_using_bert()

In [ ]:
import pandas as pd
import numpy as np
 
df = pd.read_csv("/content/drive/My Drive/corpus/test_corpus_using_livedoor_dataset.csv")

In [ ]:
from sklearn.model_selection import train_test_split
# 学習 : テスト = 95% : 5% となるようにデータを分割
train_df, test_df = train_test_split(df, test_size=0.05, stratify=df["Label"])

In [ ]:
corpus = train_df["Text"].to_list()
target_sentences = test_df["Text"].to_list()[:5]

In [ ]:
compute_similarity(model=model, corpus=corpus, 
                   target_sentences=target_sentences, recommend_sentences=5)





Target sentence : 世界初！ F1が音楽フェス会場を疾走する！　震災の復興支援のためチャリティー・ラン開催！世界的DJであるFATBOY SLIM公認のビーチパーティー「BIG BEACH FESTIVAL」が今年も2011年6月4日（土）千葉県幕張海浜公園にて開催される。大好評だった昨年に引き続き、今回で3度目の夏フェスとなる。「BIG BEACH FESTIVAL」とは、英国ブライトンにおいて25万人ものオーディエンスを熱狂させた伝説のモンスターフェスティバルの日本版であり、ダンスミュージック界の大御所FATBOY SLIM が日本で唯一認めたオフィシャル・ビーチパーティーだ。約2万人を動員した前回に続き、今回は装いも新たに多彩なDJ軍団が集結。大注目DJは、意外にも日本の野外フェス初出演のSTEVE LAWLER。STEVE LAWLERは、日本でも絶大な人気を誇る。マイアミやイビサなど開放的なロケーションでのDJプレイに定評ある彼だけに、国内初野外フェス出演は見逃せない。「BIG BEACH STAGE」と「DIESEL ISLAND STAGE」の2つのステージでDJを披露してくれる。BIG BEACH FESTIVAL'09 の模様また、今年は毎年恒例となっている花火やフライトパフォーマンスに続き、新たなエンターテインメントが仲間入りすることとなった。それはなんと、音楽フェス会場を疾走するF1カー！2010年F1世界選手権においてダブルタイトルを獲得したRed Bull Racingチームが東日本大震災の復興支援の一環として、会場内を疾走するという。世界初の試みであるから、ミュージックフリークはもちろん、全世界のモーター スポーツ・ファンが大注目するイベントとなるだろう。さらに、福島にホームベースを置くRed Bull アスリートの室屋義秀とチームメイトであるユルギス・カイリスによる2機でのフライトパフォーマンスが、14:15と18:00の2回開催される。室屋は2009年に開催された同イベントでもをエアロバティックなパフォーマンスで会場を盛り上げた。今回は飛行訓練を行っている、ふくしまスカイパークが被災したものの「このフェスに参加するために5月から再び態勢を整えて準備してきました。さずけて頂いた翼で、みなさんに世界最高峰のパフォー